In [1]:
import os, sys
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.mongodb.spark:mongo-spark-connector_2.11:2.4.0 --jars /usr/local/jar_files/spark-streaming-kafka-0-8-assembly_2.11-2.3.2.jar pyspark-shell'

In [2]:
# Import findspark 
import findspark

# Initialize and provide path
findspark.init("/usr/share/spark/spark-2.3.2-bin-hadoop2.7/")

In [3]:
#    Spark
from pyspark import SparkContext
#    Spark Streaming
from pyspark.streaming import StreamingContext
#    Kafka
from pyspark.streaming.kafka import KafkaUtils
#    json parsing
import json
#    pprint
import pprint

In [4]:
# Import SparkSession
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("master_consumer") \
    .config("spark.mongodb.input.uri", "mongodb://localhost/stocks_db.stock_coll") \
    .config("spark.mongodb.output.uri", "mongodb://localhost/stocks_db.stock_coll") \
    .getOrCreate()

In [8]:
# create the master dataframe from the existing MongoDB database
output = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri",
"mongodb://localhost/stocks_db.stock_coll").load()
output.show(truncate=False)

+--------------------------+----------+-----------------+------------------+------------------+----------+------+------------------------+
|_id                       |last_price|optionPrice_atm  |optionPrice_itm   |optionPrice_otm   |prev_price|ticker|updated_on              |
+--------------------------+----------+-----------------+------------------+------------------+----------+------+------------------------+
|[5e3dc4931bc8b1d3421dd0b9]|319.12    |7.556598994455783|21.47261698396443 |1.5036856746231564|319.04    |AAPL  |2020-02-07T20:11:38.502Z|
|[5e3dc4931bc8b1d3421dd0ba]|318.98    |7.553283865791883|21.47078240880245 |1.5017551015285662|318.97    |AAPL  |2020-02-07T20:11:16.144Z|
|[5e3dc4931bc8b1d3421dd0bb]|319.04    |7.554704635219281|21.471568537716678|1.5025823808579801|318.98    |AAPL  |2020-02-07T20:11:27.532Z|
|[5e3dc4971bc8b1d3421dd0bc]|319.12    |7.556598994455783|21.47261698396443 |1.5036856746231564|319.12    |AAPL  |2020-02-07T20:11:53.526Z|
|[5e3dc49f1bc8b1d3421dd0be]

In [9]:
output.describe().show()

+-------+------------------+-----------------+------------------+------------------+------------------+------+--------------------+
|summary|        last_price|  optionPrice_atm|   optionPrice_itm|   optionPrice_otm|        prev_price|ticker|          updated_on|
+-------+------------------+-----------------+------------------+------------------+------------------+------+--------------------+
|  count|               261|              261|               261|               261|               261|   261|                 261|
|   mean|350.11047892720325|8.290437744452683|22.047566685117218| 2.087303765980238| 350.1055363984677|  null|                null|
| stddev|110.68572629322276|2.620981599423343| 2.054161855621807|2.0850718541087314|110.67588307452777|  null|                null|
|    min|            318.25|7.535997837758686|21.461231990585134|1.4917030046260606|            318.25|  AAPL|2020-02-07T20:11:...|
|    max|            744.96|17.64027321042181|29.377021682197665| 9.52691234